# About Dataset

Dataset is downloaded from this kaggle link: https://www.kaggle.com/datasets/meetnagadia/amazon-kindle-book-review-for-sentiment-analysis?resource=download

This is a small subset of dataset of Book reviews from Amazon Kindle Store category.

### Content
5-core dataset of product reviews from Amazon Kindle Store category from May 1996 - July 2014. Contains total of 982619 entries. Each reviewer has at least 5 reviews and each product has at least 5 reviews in this dataset.

### Columns

- **asin** - ID of the product, like B000FA64PK
- **helpful** - helpfulness rating of the review - example: 2/3.
- **overall** - rating of the product.
- **reviewText** - text of the review (heading).
- **reviewTime** - time of the review (raw).
- **reviewerID** - ID of the reviewer, like A3SPTOKDG7WBLN
- **reviewerName** - name of the reviewer.
- **summary** - summary of the review (description).
- **unixReviewTime** - unix timestamp.

### Which file to use?
There are two files one is preprocessed ready for sentiment analysis and other is unprocessed to you basically have to process the dataset and then perform sentiment analysis

### Inspiration
- Sentiment analysis on reviews.
- Understanding how people rate usefulness of a review/ What factors influence helpfulness of a review.
- Fake reviews/ outliers.
- Best rated product IDs, or similarity between products based on reviews alone (not the best idea ikr).
- Any other interesting analysis

## Steps followed in this project:
1. Data preprocessing and cleaning (Feature Engineering)
2. Train-test split
3. BOW, TFIDF, Word2Vec
4. Train ML algorithms

In [1]:
# importing the library and load the dataset

import numpy as np
import pandas as pd

data = pd.read_csv('all_kindle_review .csv')
data.head()


,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [2]:
# only 'ratingtext' and 'rating' columns are required for this project
df = data[['reviewText','rating']]
df

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4
...,...,...
11995,Valentine cupid is a vampire- Jena and Ian ano...,4
11996,I have read all seven books in this series. Ap...,5
11997,This book really just wasn't my cuppa. The si...,3
11998,"tried to use it to charge my kindle, it didn't...",1


In [3]:
df.shape

(12000, 2)

In [4]:
# information
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reviewText  12000 non-null  object
 1   rating      12000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 187.6+ KB


In [5]:
# checking missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
# unique ratings
df['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [7]:
# dataset has balanced rating counts or not
df['rating'].value_counts()

5    3000
4    3000
3    2000
2    2000
1    2000
Name: rating, dtype: int64

## Preprocessing and cleaning

In [8]:
# if rating < 3: negative review or 0
# if rating > 3: positive review or 1
df['review'] = df['rating'].apply(lambda x:0 if x<3 else 1)
df.head()

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\159144290.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['review'] = df['rating'].apply(lambda x:0 if x<3 else 1)


,reviewText,rating,review
0,"Jace Rankin may be short, but he's nothing to ...",3,1
1,Great short read. I didn't want to put it dow...,5,1
2,I'll start by saying this is the first of four...,3,1
3,Aggie is Angela Lansbury who carries pocketboo...,3,1
4,I did not expect this type of book to be in li...,4,1


In [9]:
df['review'].value_counts()

1    8000
0    4000
Name: review, dtype: int64

In [10]:
# lower all the cases
df['reviewText'] = df['reviewText'].str.lower()
df.head()

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\3625730169.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].str.lower()


,reviewText,rating,review
0,"jace rankin may be short, but he's nothing to ...",3,1
1,great short read. i didn't want to put it dow...,5,1
2,i'll start by saying this is the first of four...,3,1
3,aggie is angela lansbury who carries pocketboo...,3,1
4,i did not expect this type of book to be in li...,4,1


In [11]:
# removing special characters
import re
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+','',x))

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\3697576600.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+','',x))


In [12]:
# removing the stopwords
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

df['reviewText'] = df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Madhu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\2097264871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))


In [13]:
# remove url
df['reviewText'] = df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_]+(?:(?:\.[\w_]+)+))([\w_.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\3296841254.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:re.sub(r'(http|https|ftp|ssh)://([\w_]+(?:(?:\.[\w_]+)+))([\w_.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?','',str(x)))


In [14]:
# remove html tags
from bs4 import BeautifulSoup

df['reviewText'] = df['reviewText'].apply(lambda x:BeautifulSoup(x, 'lxml').get_text())

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\3824178988.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:BeautifulSoup(x, 'lxml').get_text())


In [15]:
# remove any additional spaces
df['reviewText'] = df['reviewText'].apply(lambda x:" ".join(x.split()))

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\2537468906.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:" ".join(x.split()))


In [16]:
df.head()

,reviewText,rating,review
0,jace rankin may short hes nothing mess man hau...,3,1
1,great short read didnt want put read one sitti...,5,1
2,ill start saying first four books wasnt expect...,3,1
3,aggie angela lansbury carries pocketbooks inst...,3,1
4,expect type book library pleased find price right,4,1


In [17]:
# lemmatize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [18]:
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))

C:\Users\Madhu\AppData\Local\Temp\ipykernel_10360\1005483539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['reviewText'] = df['reviewText'].apply(lambda x:lemmatize_words(x))


In [19]:
df.head()

,reviewText,rating,review
0,jace rankin may short he nothing mess man haul...,3,1
1,great short read didnt want put read one sitti...,5,1
2,ill start saying first four book wasnt expecti...,3,1
3,aggie angela lansbury carry pocketbook instead...,3,1
4,expect type book library pleased find price right,4,1


## Train Test Split

In [20]:
X = df['reviewText']
Y = df['review']

In [21]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest = train_test_split(X,Y,test_size = 0.2)

In [22]:
print('xtrain:',xtrain.shape)
print('ytrain:',ytrain.shape)
print('xtest:',xtest.shape)
print('ytest:',ytest.shape)

xtrain: (9600,)
ytrain: (9600,)
xtest: (2400,)
ytest: (2400,)


## BOW, TFIDF, Word2Vec
Convert sentences into vectors

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=10000)

In [24]:
xtrain_bow = bow.fit_transform(xtrain).toarray()
xtest_bow = bow.transform(xtest).toarray()

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(max_features=10000)

xtrain_tfidf = tfidf.fit_transform(xtrain).toarray()
xtest_tfidf = tfidf.transform(xtest).toarray()

In [26]:
xtrain_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [27]:
print('xtrain:',xtrain.shape)
print('xtrain_bow:',xtrain_bow.shape)
print('xtrain_tfidf:',xtrain_tfidf.shape)

xtrain: (9600,)
xtrain_bow: (9600, 10000)
xtrain_tfidf: (9600, 10000)


## Train ML algorithms

In [28]:
# Naive Bayes algorithm
from sklearn.naive_bayes import GaussianNB
nb_bow =  GaussianNB().fit(xtrain_bow,ytrain)
nb_tfidf = GaussianNB().fit(xtrain_tfidf,ytrain)

In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [30]:
ypred_bow = nb_bow.predict(xtest_bow)
ypred_tfidf = nb_tfidf.predict(xtest_tfidf)

In [31]:
print('accuracy score of BOW :',accuracy_score(ytest,ypred_bow)*100)
print('accuracy score of TFIDF :',accuracy_score(ytest,ypred_tfidf)*100)

accuracy score of BOW : 54.333333333333336
accuracy score of TFIDF : 57.833333333333336


In [32]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
rf_bow = RandomForestClassifier().fit(xtrain_bow,ytrain)
rf_tfidf = RandomForestClassifier().fit(xtrain_tfidf,ytrain)

In [33]:
ypred_rf_bow = rf_bow.predict(xtest_bow)
ypred_rf_tfidf = rf_tfidf.predict(xtest_tfidf)

In [34]:
print('accuracy score of BOW with Randomforest classifier :',accuracy_score(ytest,ypred_rf_bow)*100)
print('accuracy score of TFIDF with Randomforest classifier :',accuracy_score(ytest,ypred_rf_tfidf)*100)

accuracy score of BOW with Randomforest classifier : 79.375
accuracy score of TFIDF with Randomforest classifier : 77.70833333333333


Accuracy increased by 25% with Random Forest Classifier algorithm. Using word2vec or avgword2vec techniques might increase the accuracy score a bit as well. 